In [1]:
import pyspark
from pyspark import SparkContext
lines = sc.textFile("adj_noun_pairs.txt")

In [8]:
a=sc.parallelize(xrange(1,1000))
t=100
v=a.filter(lambda x: x*x<t)
print(v.collect())
t=200
c=v.filter(lambda x: x*x<t)          
c.collect()



[1, 2, 3, 4, 5, 6, 7, 8, 9]


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [2]:
print lines.count()

3162692


In [3]:
lines.getNumPartitions()

2

In [4]:
lines.take(5)

[u'early radical',
 u'french revolution',
 u'pejorative way',
 u'violent means',
 u'positive label']

In [5]:
# Converting lines into word pairs. 
# Data is dirty: some lines have more than 2 words, so filter them out.
lines = sc.textFile("adj_noun_pairs.txt")
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()
#pairs.collect()

PythonRDD[6] at RDD at PythonRDD.scala:53

In [6]:




pairs.take(5)

[(u'early', u'radical'),
 (u'french', u'revolution'),
 (u'pejorative', u'way'),
 (u'violent', u'means'),
 (u'positive', u'label')]

In [7]:
N = pairs.count()

In [8]:
N

3162674

In [9]:
# Compute the frequency of each pair.
# Ignore pairs that not frequent enough
pair_freqs = pairs.map(lambda p:  (p,1)).reduceByKey(lambda f1, f2: f1 + f2) \
                  .filter(lambda pf: pf[1] >= 100)

In [10]:
pair_freqs.take(5)

[((u'much', u'debate'), 136),
 ((u'new', u'name'), 221),
 ((u'other', u'country'), 1857),
 ((u'other', u'book'), 223),
 ((u'present', u'husband'), 414)]

In [11]:
# Computing the frequencies of the adjectives and the nouns
a_freqs = pairs.map(lambda p: (p[0],1)).reduceByKey(lambda x,y: x+y)
n_freqs = pairs.map(lambda p: (p[1],1)).reduceByKey(lambda x,y: x+y)

In [12]:
a_freqs.take(5)

[(u'fawn', 2),
 (u'base-paired', 3),
 (u'eicosapentanoic', 1),
 (u'host-cell', 2),
 (u'1,800', 1)]

In [13]:
n_freqs.count()

106333

In [14]:
# Broadcasting the adjective and noun frequencies. 
#a_dict = a_freqs.collectAsMap()
#a_dict = sc.parallelize(a_dict).map(lambda x: x)
n_dict = sc.broadcast(n_freqs.collectAsMap())
a_dict = sc.broadcast(a_freqs.collectAsMap())
global b
b=a_freqs.collectAsMap()
print a_dict.value['violent']
print b['violent']

1191
1191


In [15]:
 from math import *

# Computing the PMI for a pair.
def pmi_score(pair_freq):
    w1, w2 = pair_freq[0]
    f = pair_freq[1]
    pmi = log(float(f)*N/(b[w1]*n_dict.value[w2]), 2)
    return pmi, (w1, w2)

In [16]:
# Computing the PMI for all pairs.
scored_pairs = pair_freqs.map(pmi_score)

In [17]:
# Printing the most strongly associated pairs. 
scored_pairs.top(10)

[(14.41018838546462, (u'magna', u'carta')),
 (13.071365888694997, (u'polish-lithuanian', u'Commonwealth')),
 (12.990597616733414, (u'nitrous', u'oxide')),
 (12.64972604311254, (u'latter-day', u'Saints')),
 (12.50658937509916, (u'stainless', u'steel')),
 (12.482331020687814, (u'pave', u'runway')),
 (12.19140721768055, (u'corporal', u'punishment')),
 (12.183248694293388, (u'capital', u'punishment')),
 (12.147015483562537, (u'rush', u'yard')),
 (12.109945794428935, (u'globular', u'cluster'))]